![](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F1095143%2F47c74960e6540f11287e1e271438e029%2FTReNDS.png?generation=1587603283379241&alt=media)

- <a href='#1'>1. Introdução</a>  
- <a href='#2'>2. Carregar dados</a>
     - <a href='#2-1'>2.1 Carregar bibliotecas</a>
     - <a href='#2-2'>2.2 Ler os dados</a>
- <a href='#3'>3. Visualização dos dados</a>
     - <a href='#3-1'>3.1 Visão geral das tabelas</a>
     - <a href='#3-2'>3.2 Visão geral estatística dos dados</a>
- <a href='#4'>4. Verificação de dados ausentes</a>
- <a href='#5'>5. Exploração de dados</a>
    - <a href='#5-1'>5.1 Distribuição de variáveis de entrada em loading_data</a>
    - <a href='#5-2'>5.2 Distribuição das variáveis-alvo nos dados do treino</a>
    - <a href='#5-3'>5.3 Explorar mapas espaciais de fMRI</a>
        - <a href='5-3-1'> 5.3.1 Exibindo mapas de componentes individuais</a>
- <a href='#6'>6. Amostras de envios (submissions)</a>
    - <a href='#6-1'>6.1 Baseline submission</a>
    - <a href='#6-2'>6.2 Média de submission</a>

 # <a id='1'> 1. Introdução </a>
A pesquisa do cérebro humano está entre as áreas de estudo mais complexas para os cientistas. Sabemos que a idade e outros fatores podem afetar sua função e estrutura, mas são necessárias mais pesquisas sobre o que ocorre especificamente no cérebro. Com grande parte da pesquisa usando exames de ressonância magnética, os cientistas de dados estão bem posicionados para apoiar idéias futuras. Em particular, especialistas em neuroimagem procuram marcadores mensuráveis ​​de comportamento, saúde ou distúrbio para ajudar a identificar regiões cerebrais relevantes e sua contribuição para efeitos típicos ou sintomáticos.


Nesta competição, você irá prever várias avaliações mais a idade dos recursos de ressonância magnética cerebral multimodal. Você estará trabalhando com os resultados existentes de outros cientistas de dados, realizando o importante trabalho de validar a utilidade dos recursos multimodais em uma população normativa de indivíduos não afetados. Devido à complexidade do cérebro e às diferenças entre os scanners, abordagens generalizadas serão essenciais para impulsionar efetivamente a pesquisa de neuroimagem multimodal adiante.


![](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F1537731%2Fa5fdbe17ca91e6713d2880887232c81a%2FScreen%20Shot%202019-12-09%20at%2011.25.31%20AM.png?generation=1575920121028151&alt=media)


O Tri-Institucional Universidade Estadual da Geórgia / Instituto de Tecnologia da Geórgia / Centro Emory University de Pesquisa Translacional em Neuroimagem e Ciência de Dados (TReNDS) utiliza imagens avançadas do cérebro para promover pesquisas sobre a saúde do cérebro. A organização está focada no desenvolvimento, aplicação e compartilhamento de abordagens analíticas avançadas e ferramentas de neuroinformática. Entre seus projetos de software estão as caixas de ferramentas de neuroimagem GIFT e FIT, o sistema de gerenciamento de dados COINS e o kit de ferramentas COINSTAC para aprendizado federado, todos destinados a apoiar cientistas de dados e outros pesquisadores de neuroimagem.

Dar o salto da pesquisa para a aplicação clínica é particularmente difícil na saúde do cérebro. Para traduzir para contextos clínicos, os resultados da pesquisa devem ser reproduzidos de forma consistente e validados em casos fora da amostra. O problema é particularmente adequado para a ciência de dados, mas as abordagens atuais geralmente não se generalizam. Com esse grande conjunto de dados e competição, seus esforços podem abordar diretamente uma área importante da pesquisa do cérebro.

# <a id='2'> 2. Carregar dados </a>

## <a id='2-1'> 2.1 Carregar bibliotecas </a>

In [ ]:
import pandas as pd 
import numpy as np  
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns           
color = sns.color_palette()
import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode()

import cufflinks as cf
cf.go_offline()

# Venn diagram
from matplotlib_venn import venn2
import re
import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
eng_stopwords = stopwords.words('english')
import gc

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import os
import nilearn as nl

In [ ]:
path = '/kaggle/input/trends-assessment-prediction'
print(os.listdir(path))

## <a id='2-1'> 2.2 Ler os dados</a>

In [ ]:
df_loading           = pd.read_csv(path +'/loading.csv')
df_train             = pd.read_csv(path +'/train_scores.csv')
df_sample_submission = pd.read_csv(path +'/sample_submission.csv')

In [ ]:
print('Tamanho de df_loading', df_loading.shape)
print('Tamanho de df_train', df_train.shape)
print('Tamanho de sample_submission', df_sample_submission.shape)
print('test tamanho:', len(df_sample_submission)/5)


# <a id='3'> 3. Visualização dos dados</a>

In [ ]:
df_train.head()

In [ ]:
df_train.info()

Observamos que temos valores nulos nas variáveis: 
- domain1_var1 
- domain1_var2
- domain2_var1
- domain2_var2 

In [ ]:
df_sample_submission.head()

## <a id='3-1'> 3.1 Visão geral das tabelas</a>

In [ ]:
display(df_loading.head())
display(df_loading.describe())

**df_train_data**
> ##### Na verdade, são apenas os valores de destino para o conjunto de dados de treinamento. Os dados reais de treinamento estão no carregamento de dados (parcialmente). Então, vou me concentrar apenas nesse arquivo.

In [ ]:
display(df_train.head())
display(df_train.describe())

**df_sample_submission**

In [ ]:
df_sample_submission.head()

**Variáveis alvo**

In [ ]:
targets = list(df_train.columns[1:])
targets

## <a id='3-2'> 3.2 Visão geral estatística dos dados</a>

**Verificação de dados ausentes em df_train**

In [ ]:
# checking missing data
total   = df_train.isnull().sum().sort_values(ascending = False)
percent = (df_train.isnull().sum() / train_data.isnull().count() * 100).sort_values(ascending = False)

In [ ]:
df_missing_train_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
df_missing_train_data.head()

**Verificação de dados ausentes em df_loading**

In [ ]:
# checking missing data
total   = df_loading.isnull().sum().sort_values(ascending = False)
percent = (df_loading.isnull().sum() / df_loading.isnull().count()*100).sort_values(ascending = False)

df_missing_test  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
df_missing_test.head()

## <a id='5'> 5. Exploração de dados</a>

In [ ]:
def distributionTarget(df, type):

    targets   = df.columns[1:]
    fig, axes = plt.subplots(6, 5, figsize=(18, 15))
    fig, axes = plt.subplots(1, 5, figsize=(18, 4))
   
    axes = axes.ravel()
    
    if type == 1:
        bins = np.linspace(-0.05, 0.05, 20) 
    else: 
        bins = np.linspace(0, 100, 20) 

    for i, col in enumerate(targets):
        ax = axes[i]
        sns.distplot(df[col], label = col, kde = False, bins = bins, ax = ax)

    plt.tight_layout()
    plt.show()
    plt.close()

## <a id='5-1'> 5.1 Distribuição de variáveis de entrada em loading_data</a>

In [ ]:
distributionTarget(df_loading, 1)

## <a id='5-2'> 5.2 Distribuição de variáveis de entrada em df_loading</a>

In [ ]:
distributionTarget(df_train, 2)

## <a id='5-3'> 5.3 Explorar mapas espaciais de fMRI </a>
Em geral, os mapas espaciais são salvos como tensores 4-D

$$\mathcal{X}_i \in \mathbb{R}^{X \times Y \times Z \times K}$$

onde $ X $, $ Y $ e $ Z $ são as três dimensões espaciais do volume e $ K $ é o número de componentes.

## Formato de arquivo
Os mapas espaciais do assunto foram salvos em arquivos `.mat` usando o sinalizador` v7.3`; portanto, eles devem ser carregados como conjuntos de dados `h5py`,e um arquivo nifti deve ser usado para definir os cabeçalhos para fins de exibição. Incluímos a função `load_subject`, que leva um nome de arquivo `.mat` do assunto e a imagem nilearn carregada a ser usada para definir os cabeçalhos.

In [ ]:
def load_subject(filename, mask_niimg):
    """
    Carrega os dados salvo no formato .mat com
         o sinalizador da versão 7.3. Retornar os dados 
         em niimg, usando uma máscara niimg como modelo
         para cabeçalhos nifti.
        
     Args:
         filename <str> o nome do arquivo .mat para os dados do 
         objeto mask_niimg niimg o objeto mask niimg usado para cabeçalhos nifti         
    """
    subject_data = None

    with h5py.File(subject_filename, 'r') as f:
        subject_data = f['SM_feature'][()]
        
    # É necessário reorientar os eixos, pois o h5py vira a ordem dos eixos
    
    subject_data  = np.moveaxis(subject_data, [0,1,2,3], [3,2,1,0])
    subject_niimg = nl.image.new_img_like(mask_niimg, subject_data, affine=mask_niimg.affine, copy_header=True)
    
    return subject_niimg

In [ ]:
path_fMRI_mat = '/kaggle/input/trends-assessment-prediction/fMRI_train/'

In [ ]:
import h5py
import nilearn.plotting as nlplt

In [ ]:
mask_filename    = '/kaggle/input/trends-assessment-prediction/fMRI_mask.nii'
subject_filename = '../input/trends-assessment-prediction/fMRI_train/10015.mat'
smri_filename    = 'ch2better.nii'
mask_niimg       = nl.image.load_img(mask_filename)

In [ ]:
subject_niimg = load_subject(subject_filename, mask_niimg)

In [ ]:
print("Image shape is %s" % (str(subject_niimg.shape)))
num_components = subject_niimg.shape[-1]
print("Detected {num_components} spatial maps".format(num_components=num_components))


 ## Download da imagem do modelo ch2better para exibição

In [ ]:
!wget https://github.com/Chaogan-Yan/DPABI/raw/master/Templates/ch2better.nii

## Exibindo todos os componentes em um atlas de probabilidades
Primeiro, exibiremos os 53 mapas espaciais em um atlas completo usando a função nilearn `plot_prob_atlas`. Estes
os mapas serão sobrepostos em um modelo estrutural de RM.

In [ ]:
nlplt.plot_prob_atlas(subject_niimg, 
                      bg_img     = smri_filename, 
                      view_type  = 'filled_contours', 
                      draw_cross = False, 
                      title      = 'All %d spatial maps' % num_components, 
                      threshold  = 'auto')


## <a id='5-3-1'> 5.3.1 Exibindo mapas de componentes individuais</a>
Além disso, podemos exibir separadamente cada um dos 53 mapas para obter uma visão mais completa
da estrutura de componentes individuais.

In [ ]:
grid_size = int(np.ceil(np.sqrt(num_components)))
fig, axes = plt.subplots(grid_size, grid_size, figsize=(grid_size*10, grid_size*10))
[axi.set_axis_off() for axi in axes.ravel()]



for i, cur_img in enumerate(nl.image.iter_img(subject_niimg)):
    col = i % grid_size
    if col == 0:
        row += 1
    nlplt.plot_stat_map(cur_img, bg_img=smri_filename, title="IC %d" % i, axes=axes[row, col], threshold=3, colorbar=False)